In [21]:
import pandas as pd
import nltk
import gensim.downloader
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import numpy as np

df_books = pd.read_csv("Books.csv")

/var/folders/v2/rwg0t7796jsbrq_0j88y5mk40000gn/T/ipykernel_83825/4128192709.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv("Books.csv")


In [2]:
df_books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [3]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

In [5]:
glove_vectors.most_similar('twitter')

[('Twitter', 0.8908904194831848),
 ('Twitter.com', 0.7536780834197998),
 ('tweet', 0.7431627511978149),
 ('tweeting', 0.7161932587623596),
 ('tweeted', 0.7137226462364197),
 ('facebook', 0.6988551616668701),
 ('tweets', 0.6974530220031738),
 ('Tweeted', 0.6950210332870483),
 ('Tweet', 0.6875007152557373),
 ('Tweeting', 0.6845167279243469)]

In [6]:
df_ratings = pd.read_csv('Ratings.csv')
df_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [7]:
stop_words = set(stopwords.words('english'))

In [24]:
title_dict = {}
for row in df_books.head(100).index:
    title = df_books['Book-Title'][row]
    title_nonum = re.sub(r'\d+', '', title) #remove number
    tokenizer = RegexpTokenizer(r'\w+') #remove punctuation
    title_tokens = tokenizer.tokenize(title_nonum) 
    filtered_title_tokens = [w for w in title_tokens if not w.lower() in stop_words] #remove stop words
    title_vectors = []
    for title_token in filtered_title_tokens:
        try:
            title_token_vec = glove_vectors[title_token]
        except:
            title_token_vec = np.zeros(300)
        title_vectors.append(title_token_vec)
    title_dict[title] = title_vectors
print(title_dict)

{'Classical Mythology': [array([ 6.78710938e-02,  8.78906250e-02,  1.67236328e-02, -2.20703125e-01,
        2.09960938e-01, -2.30712891e-02,  2.00195312e-01,  5.15136719e-02,
        8.83789062e-02,  6.25000000e-02, -2.91015625e-01, -9.32617188e-02,
        1.15234375e-01,  1.66992188e-01,  6.68945312e-02,  1.07910156e-01,
        1.69921875e-01,  5.95092773e-03, -9.71679688e-02, -2.06054688e-01,
        7.47070312e-02,  1.82617188e-01,  1.02539062e-02, -7.42187500e-02,
        2.53906250e-02,  3.57421875e-01,  5.76782227e-03, -1.57226562e-01,
       -3.28125000e-01, -9.37500000e-02, -8.15429688e-02, -3.85742188e-02,
       -1.99218750e-01,  5.79833984e-03, -7.32421875e-03, -3.59375000e-01,
       -1.27929688e-01, -2.57812500e-01, -1.96289062e-01,  2.90527344e-02,
       -3.05175781e-02,  3.56445312e-02,  5.32226562e-02,  1.97265625e-01,
        1.89453125e-01,  1.37695312e-01, -1.71875000e-01,  2.01171875e-01,
       -7.29370117e-03,  8.98437500e-02, -1.41601562e-01,  3.57421875e-01,
